In [24]:
import onnx
import torch
import numpy as np
from onnx import numpy_helper as nh
import copy
import pickle
import hashlib
from onnx_functions import OnnxFunctions as OnnxModel
from difftools import *
import sys

In [3]:
class MLP(torch.nn.Module):
        def __init__(self):
            super(MLP, self).__init__()
            self.fc1 = torch.nn.Linear(5, 3)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(3, 1)
            self.sigmoid = torch.nn.Sigmoid()

        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

dummy_input = torch.rand(5)
model = MLP()

torch.onnx.export(model, dummy_input, "model.onnx")

In [4]:
onnx_model = OnnxModel("model.onnx")
onnx_model.get_input_output_weight_names()

{'inputs': ['input.1'],
 'outputs': ['12'],
 'weights': ['13', '14', 'fc1.bias', 'fc2.bias']}

In [7]:
onnx_model.get_weight_by_name('fc1.bias')

dims: 3
data_type: 1
name: "fc1.bias"
raw_data: "\210\021\352=@>\322<\200\313\260>"

In [5]:
# Hand-written
name_map = {
    "12": "output",
    "13": "weights1",
    "14": "weights2",
    "6" : "matmul0_output",
    "7" : "add1_output",
    "8" : "relu2_output",
    "10": "matmul3_output",
    "11": "add4_output"
}

onnx_model.change_names(name_map)

onnx_model.get_input_output_weight_names()

{'inputs': ['input.1'],
 'outputs': ['output'],
 'weights': ['weights1', 'weights2', 'fc1.bias', 'fc2.bias']}

## Diff 101 - Creating a diff file, and applying it to an ONNX model

In [43]:
del sys.modules["difftools"]
from difftools import *

update_matrix = torch.rand((5,3))

print('Update Matrix:\n', update_matrix, '\n---\n')

basic_update = ModelUpdate(initializer_name="weights1", update_type="dense", value=update_matrix)

diffname = create_diff_file(basic_update)

Update Matrix:
 tensor([[0.3148, 0.0590, 0.6038],
        [0.6837, 0.8210, 0.4121],
        [0.6220, 0.3701, 0.8425],
        [0.2072, 0.9407, 0.2001],
        [0.9929, 0.3732, 0.0429]]) 
---



In [44]:
del sys.modules["difftools"]
from difftools import *

updated_onnx_model = apply_diff_file(onnx_model, diffname)

print('Initializer in the original model: \n', nh.to_array(onnx_model.get_weight_by_name("weights1")))
print('\n---\n')
print('Initializer in the updated model: \n', nh.to_array(updated_onnx_model.get_weight_by_name("weights1")))

Initializer in the original model: 
 [[ 0.40592372  0.23077822  0.4108742 ]
 [ 0.19051802  0.15537947 -0.17507884]
 [-0.28606987 -0.34657827 -0.39699313]
 [-0.2849418  -0.22728954 -0.44549283]
 [ 0.1846577   0.30758405 -0.38323548]]

---

Initializer in the updated model: 
 [[0.3147961  0.05896282 0.6037595 ]
 [0.68366444 0.820969   0.4121133 ]
 [0.62203926 0.37009144 0.84248894]
 [0.2071746  0.9407131  0.20005661]
 [0.99287033 0.3731795  0.04293466]]


## Updating multiple initializers in a single go

In [50]:
del sys.modules["difftools"]
from difftools import *

update_matrix_1 = torch.rand((5,3))
update_matrix_2 = torch.rand((3,1))

print('Update Matrix 1:\n', update_matrix_1, '\n---\n')
print('Update Matrix 2:\n', update_matrix_2, '\n---\n')

updates = [
    ModelUpdate("weights1", "dense", update_matrix_1),
    ModelUpdate("weights2", "dense", update_matrix_2)
]

diffname = create_diff_file(updates)

multi_updated_onnx_model = apply_diff_file(onnx_model, diffname)

print('Initializers in the original model: \n', nh.to_array(onnx_model.get_weight_by_name("weights1")), '\n\n', nh.to_array(onnx_model.get_weight_by_name("weights2")))
print('\n---\n')

Update Matrix 1:
 tensor([[0.9940, 0.1585, 0.5711],
        [0.6978, 0.9266, 0.2635],
        [0.4459, 0.5328, 0.5981],
        [0.3354, 0.1249, 0.9643],
        [0.0641, 0.0722, 0.5938]]) 
---

Update Matrix 2:
 tensor([[0.7789],
        [0.4640],
        [0.3771]]) 
---

2
Initializers in the original model: 
 [[ 0.40592372  0.23077822  0.4108742 ]
 [ 0.19051802  0.15537947 -0.17507884]
 [-0.28606987 -0.34657827 -0.39699313]
 [-0.2849418  -0.22728954 -0.44549283]
 [ 0.1846577   0.30758405 -0.38323548]] 

 [[ 0.22380811]
 [-0.5261495 ]
 [-0.03400868]]

---



In [51]:
print('Initializers in the updated model: \n', nh.to_array(multi_updated_onnx_model.get_weight_by_name("weights1")), '\n\n', nh.to_array(multi_updated_onnx_model.get_weight_by_name("weights2")))

Initializers in the updated model: 
 [[0.99400395 0.15845329 0.57113624]
 [0.69781685 0.9266338  0.26351333]
 [0.44593602 0.5327525  0.5981272 ]
 [0.3353712  0.12488973 0.9642594 ]
 [0.0640893  0.07219303 0.5937671 ]] 

 [[0.7789321 ]
 [0.4639855 ]
 [0.37706804]]
